# 将模型创建API服务

## 参考

<div id="refer-anchor-1"></div>

- [1] [为你的机器学习模型创建API服务](https://www.cnblogs.com/wkang/p/9905444.html)

## 创建一个简单模型

以一个kaggle经典的比赛项目：泰坦尼克号生还者预测为例，训练一个简单的模型。<br />
首先，我们需要导入训练集并选择特征。因为本文主要是介绍机器学习模型API的编写，所以模型训练过程并不做为重点内容，因此我们只选择其中的'Age', 'Sex', 'Embarked', 'Survived' 这四个特征来构造训练集。

In [1]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# 导入训练集并选择特征
df = pd.read_csv('input/train.csv')
include = ['Age', 'Sex', 'Embarked', 'Survived']  
df_ = df[include]

然后，是一个简单的数据处理过程。
这里主要是对类别型特征进行One-hot编码，对连续型特征进行空缺值填充。

In [2]:
categoricals = []
for col, col_type in df_.dtypes.iteritems():
    if col_type == 'O':
        categoricals.append(col)
    else:
        df_[col].fillna(0, inplace=True)

df_ohe = pd.get_dummies(df_, columns=categoricals, dummy_na=True)

最后，是模型的训练以及持久化保存。

模型采用的是逻辑回归，使用sklearn.externals.joblib将模型保存为序列化文件.pkl。需要注意的是，如果传入的请求不包含所有可能的category变量值，那么在预测时，get_dummies()生成的dataframe的列数比训练得到分类器的列数少，这会导致运行报错发生。所以在模型训练期间还需要持久化训练集One-hot后的列名列表。

In [3]:
from sklearn.linear_model import LogisticRegression
from joblib import dump

dependent_variable = 'Survived'
x = df_ohe[df_ohe.columns.difference([dependent_variable])]
y = df_ohe[dependent_variable]
lr = LogisticRegression()
lr.fit(x, y)

# 保存模型
dump(lr, 'model/model.joblib')
print("Model dumped!")

# 把训练集中的列名保存为joblib
model_columns = list(x.columns)
dump(model_columns, 'model/model_columns.joblib')
print("Models columns dumped!")

Model dumped!
Models columns dumped!


## 基于Flask框架创建API服务

使用Flask部署模型服务，需要写一个函数predict()，并完成以下两件事：

1. 当应用程序启动时，将已持久化的模型加载到内存中；
2. 创建一个API站点，该站点接受输入变量的请求后，将输入转换为适当的格式，并返回预测。

更具体地说，需要API的输入如下（一个由JSON组成的列表）：
``` json
[
    {"Age": 85, "Sex": "male", "Embarked": "S"},
    {"Age": 24, "Sex": "female", "Embarked": "C"},
    {"Age": 3, "Sex": "male", "Embarked": "C"},
    {"Age": 21, "Sex": "male", "Embarked": "S"}
]
```
而模型API的输出如下：
``` json
{
    "prediction": [0, 1, 1, 0]
}
```

``` python
import traceback
import sys

import pandas as pd
from flask import request
from flask import Flask
from flask import jsonify
from joblib import load

app = Flask(__name__)


# Your API endpoint URL would consist /predict
@app.route('/predict', methods=['POST'])
def predict():
    if lr:
        try:
            json_ = request.json
            query = pd.get_dummies(pd.DataFrame(json_))
            query = query.reindex(columns = model_columns, fill_value=0)
            prediction = list(lr.predict(query))
            return jsonify({'prediction': str(prediction)})
        except Exception:
            return jsonify({'trace': traceback.format_exc()})
    else:
        print('Train the model first')
        return 'No model here to use'


if __name__ == '__main__':

    try:
        port = int(sys.argv[1])
    except Exception:
        port = 9000
    # Load  model.joblib
    lr = load('model/model.joblib')
    print('Model loaded')
    # Load model_columns.joblib
    model_columns = load('model/model_columns.joblib')
    print('Model columns loaded')
    app.run(host='localhost', port=port, debug=True)
```

到此，我们的机器学习模型API已经创建完毕，flask_api.py 的代码部分也已构造完毕。但在进一步深入之前，让我们回顾一下之前的所有操作：

1. 加载了泰坦尼克数据集并选择了四个特征。
2. 进行了必要的数据预处理。
3. 训练了一个逻辑回归分类器模型并将其序列化。
4. 持久化训练集中的列名的列表。

使用Flask编写了一个简单的API，该API通过接收一个由JSON组成的列表，预测一个人是否在沉船中幸存。

## API的有效性测试

``` python
import requests

years_exp = [{"Age": 22, "Sex": "male", "Embarked": "S"},
             {"Age": 22, "Sex": "female", "Embarked": "C"},
             {"Age": 80, "Sex": "female", "Embarked": "C"},
             {"Age": 22, "Sex": "male", "Embarked": "S"},
             {"Age": 22, "Sex": "female", "Embarked": "C"},
             {"Age": 80, "Sex": "female", "Embarked": "C"},
             {"Age": 22, "Sex": "male", "Embarked": "S"},
             {"Age": 22, "Sex": "female", "Embarked": "C"},
             {"Age": 80, "Sex": "female", "Embarked": "C"},
             {"Age": 22, "Sex": "male", "Embarked": "S"},
             {"Age": 22, "Sex": "female", "Embarked": "C"},
             {"Age": 80, "Sex": "female", "Embarked": "C"},
             {"Age": 22, "Sex": "male", "Embarked": "S"},
             {"Age": 22, "Sex": "female", "Embarked": "C"},
             {"Age": 80, "Sex": "female", "Embarked": "C"},
             ]
response = requests.post(url='http://localhost:9000/predict', json=years_exp)
result = response.json()
print('model API返回结果：', result)
```

model API返回结果：

``` json
{'prediction': '[0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1]'}
```